In [62]:
import numpy as np
import matplotlib.pylab as plt
import networkx as nx
import pandas as pd
from scipy import optimize
import random

## Funciones útiles

In [117]:
def ldata(archive):
    f=open(archive)
    data=[]
    for line in f:
        line=line.strip()
        col=line.split()
        data.append(col)
    return data

In [118]:
def asignar_esencialidad(g):  
    escen = ldata ('tc02Data/Essential_ORFs_paperHe.txt')
    escenciales=[]
    for j in range(len(escen)):
        if len(escen[j])>1:
            escenciales.append(escen[j][1])
#     print(escenciales)    
    e=[]
    for node in g.nodes:
        if node in escenciales:
            g.node[node]['escencialidad']='escencial'
            e.append(node)
        else:
            g.node[node]['escencialidad']='no escencial'
#     print(e)
    return g

In [119]:
def grafo (archive):
    data=ldata(archive)
    g = nx.Graph ()
    g = g.to_undirected()

    for i in range (len(data)):
        g.add_nodes_from (data[i])
        g.add_edge (data[i][0],data[i][1])
    return g

#Grafos
gAP = grafo ('tc02Data/yeast_AP-MS.txt')
gLIT = grafo ('tc02Data/yeast_LIT.txt')
gY2H = grafo ('tc02Data/yeast_Y2H.txt')

Lit_Reg = ldata ('tc02Data/yeast_LIT_Reguly.txt')
Litreg = []

for i in range(1,len(Lit_Reg)):
    Litreg.append([Lit_Reg[i][0],Lit_Reg[i][1]])

gLITr = nx.Graph ()
gLITr = gLITr.to_undirected()

for i in range (len(Litreg)):
    gLITr.add_nodes_from (Litreg[i])
    gLITr.add_edge (Litreg[i][0],Litreg[i][1])

In [120]:
def nro_de_enlaces_escenciales(g):
    n = 0
    for enlace in g.edges():
        if g.node[enlace[0]]['escencialidad'] == 'escencial' and g.node[enlace[1]]['escencialidad'] == 'escencial':
            n = n + 1
    return n

In [121]:
def ordenar_nodos_por_grado(g):
    nodo_grados=[]
    for node in g.nodes:
        nodo_grados.append([node,g.degree[node]])
        if g.degree(node) == 0:
            nodo_grados.remove([node,g.degree[node]])
    nodo_grados.sort(key=lambda grado: grado[1], reverse=True)
    return nodo_grados

### Coeficiente Alfa

In [71]:
def coef_alfa(g,nro_vueltas):
    #Asigno escencialidad
    g = asignar_esencialidad(g)
    #Calculo el n de la red original (IBEP)
    n = nro_de_enlaces_escenciales(g)
    
    lista_de_m = []
    
    for _ in range(nro_vueltas):
        nuevos_edges = []
        nodo_grados = ordenar_nodos_por_grado(g)
        while len(nodo_grados) > 1: #(II)
            while nodo_grados[0][1] > 0:
                nuevo_vecino = random.choice(nodo_grados[1:])
                if [nodo_grados[0][0],nuevo_vecino[0]] not in nuevos_edges: #(III)
                    nuevo_vecino[1] = nuevo_vecino[1] - 1
                    nodo_grados[0][1] = nodo_grados[0][1] - 1
                    #mi nueva lista de enlaces puede tener menos enlaces que la original,
                    #pero no sera significativo
                    nuevos_edges.append([nodo_grados[0][0],nuevo_vecino[0]]) 
                    if nuevo_vecino[1] == 0:
                        nodo_grados.remove(nuevo_vecino)
            nodo_grados.remove(nodo_grados[0])
            #Reordeno la lista con los nodos redefinidos y con los nodos con k = 0 fuera
            nodo_grados.sort(key=lambda grado: grado[1], reverse=True) 

        #me fijo cuantos enlaces escenciales tengo
        m = 0
        for enlace in nuevos_edges:
            if g.node[enlace[0]]['escencialidad'] == 'escencial' and g.node[enlace[1]]['escencialidad'] == 'escencial':
                m = m + 1
        lista_de_m.append(m) 
    
    m = sum(lista_de_m)/len(lista_de_m)
    
    alfa = ((n-m)/(g.number_of_edges()))*100
      
    return alfa,m

In [27]:
alfaY2H,mY2H = coef_alfa(gY2H,1000)
print([alfaY2H,mY2H])

[3.3926621160409556, 176.595]


In [28]:
alfaLIT,mLIT = coef_alfa(gLIT,1000)
print([alfaLIT,mLIT])

[9.153777777777778, 788.252]


In [128]:
graf=asignar_esencialidad(gLITr)
escenciale=[]
for node in graf.nodes:
    if graf.nodes[node]['escencialidad']=='escencial':
        escenciale.append(node)
print(escenciale)

['YBR236C', 'YML010W', 'YOR257W', 'YDR167W', 'YDR145W', 'YBR198C', 'YGL112C', 'YMR236W', 'YNL172W', 'YLR127C', 'YOR249C', 'YGL099W', 'YHR170W', 'YPL093W', 'YGR103W', 'YER006W', 'YMR308C', 'YHR197W', 'YGR245C', 'YCR072C', 'YPL146C', 'YKL052C', 'YDR201W', 'YPL217C', 'YMR128W', 'YBR247C', 'YHR148W', 'YER082C', 'YNL132W', 'YCL059C', 'YJR002W', 'YPL126W', 'YDL148C', 'YLR197W', 'YPL012W', 'YMR229C', 'YPR137W', 'YGL011C', 'YBL004W', 'YDR324C', 'YGR090W', 'YGR128C', 'YJL109C', 'YLR222C', 'YMR093W', 'YCR093W', 'YDL165W', 'YKL022C', 'YDL008W', 'YDR118W', 'YBL084C', 'YHR166C', 'YMR001C', 'YER172C', 'YMR213W', 'YLR117C', 'YMR240C', 'YKL095W', 'YDL209C', 'YGR278W', 'YLR323C', 'YMR288W', 'YDL043C', 'YLL036C', 'YJL203W', 'YER013W', 'YAL032C', 'YPL151C', 'YHR165C', 'YDL030W', 'YML049C', 'YDR088C', 'YER029C', 'YGR074W', 'YLR275W', 'YLR147C', 'YPR182W', 'YKL173W', 'YOR148C', 'YDR416W', 'YGL207W', 'YML069W', 'YGL128C', 'YNL245C', 'YGL130W', 'YMR168C', 'YMR094W', 'YDR328C', 'YBR160W', 'YBR135W', 'YOR204W'

In [110]:
alfaLITr,mLITr = coef_alfa(gLITr,100)
print([alfaLITr,mLITr])

[8.256872997132737, 2372.9]


In [145]:
gLITr = asignar_esencialidad(gLITr)
nro_escenciales = 0
for node in gLITr.nodes():
    if gLITr.node[node]['escencialidad']=='escencial':
        nro_escenciales = nro_escenciales + 1
nro_escenciales

896

### Coeficiente Beta

In [141]:
def coef_beta(g,m,nro_de_vueltas):
    g = asignar_esencialidad(g)
    nro_escenciales = 0
    for node in g.nodes():
        if g.node[node]['escencialidad']=='escencial':
            nro_escenciales = nro_escenciales + 1
            
    n = nro_de_enlaces_escenciales(g)
    print(nro_escenciales)
    print(n-m)
    betas = []
    
    for node in g.nodes:
        betas.append([node,g.degree(node),0])
    print(betas)
    for _ in range(nro_de_vueltas):
        nodo_grados=[]
        for node in g.nodes:
            nodo_grados.append([node,g.degree[node]])
        random.shuffle(nodo_grados)  
        print(nodo_grados)
        #HASTA ACA OK
        #Asigno (n-m) enlaces aleatorios y escencialidad a los nodos con esos enlaces
        nuevos_enlaces = []
        i = 0
        while len(nuevos_enlaces) < int(n-m):
            nodo = random.choice(nodo_grados)
            if nodo_grados[i][0] != nodo [0] and [nodo[0],nodo_grados[i][0]] not in nuevos_enlaces:
                nuevos_enlaces.append([nodo_grados[i][0],nodo[0]])
                i = i + 1
                print(i)
        #pongo a los nodos escenciales en una lista
        nodos_escenciales=[]
        for nod in nuevos_enlaces:
            nodos_escenciales.append(nod[0])
            nodos_escenciales.append(nod[1])
        
        nodos_escenciales = list(set(nodos_escenciales))
        #pongo a los nodos  no escenciales en una lista
        print(len(nodos_escenciales))
        no_escenciales=[]

        for node in nodo_grados:
            if node[0] not in nodos_escenciales:
                no_escenciales.append(node[0])

        #Selecciono m nodos aleatoriamente y les asigno escencialidad
        random.shuffle(no_escenciales)
        print(no_escenciales)
        #Creo una lista que contenga a los nodos escenciales debido a factores que no son los enlaces escenciales
        nuevos_escenciales = []
        if len(nodos_escenciales)<nro_escenciales:
            i = 0
            while i < nro_escenciales:
                nuevos_escenciales.append(no_escenciales[i])
                i = i + 1
                print(i)
        for i in range(len(betas)): #Beta para cada nodo[i] sera sum(r)/10000
            if betas[i][0] in nuevos_escenciales:
                betas[i][2] = betas[i][2] + 1/nro_de_vueltas

    Beta = 0
    for i in range(len(betas)):
        Beta = Beta + betas[i][2]
    Beta = Beta*100/len(betas)
        
    return Beta

In [123]:
betaY2H = coef_beta(gY2H,mY2H,10000)
print(betaY2H)

22.348860257680094


In [70]:
betaLIT = coef_beta(gLIT,mLIT,10000)
print(betaLIT)

40.885416666663815


In [142]:
betaLITr = coef_beta(gLITr,mLITr,1)
print(betaLITr)

896
970.0999999999999
[['YBR236C', 4, 0], ['YML010W', 103, 0], ['YDR448W', 38, 0], ['YPL254W', 18, 0], ['YDR176W', 29, 0], ['YOR257W', 14, 0], ['YGR252W', 35, 0], ['YBR010W', 32, 0], ['YDR225W', 26, 0], ['YDR159W', 14, 0], ['YCL010C', 12, 0], ['YGL066W', 8, 0], ['YOL148C', 20, 0], ['YDR392W', 21, 0], ['YBR081C', 24, 0], ['YLR055C', 21, 0], ['YBR111W-A', 22, 0], ['YDR167W', 46, 0], ['YDR145W', 47, 0], ['YBR198C', 40, 0], ['YGL112C', 49, 0], ['YMR236W', 39, 0], ['YOL072W', 7, 0], ['YHR099W', 30, 0], ['YMR223W', 22, 0], ['YNL172W', 13, 0], ['YIR025W', 14, 0], ['YLR127C', 18, 0], ['YOR249C', 11, 0], ['YDR101C', 22, 0], ['YGL099W', 24, 0], ['YHR170W', 11, 0], ['YPL093W', 49, 0], ['YGR103W', 61, 0], ['YHR052W', 73, 0], ['YER006W', 27, 0], ['YMR308C', 33, 0], ['YHR197W', 18, 0], ['YGR245C', 15, 0], ['YBR267W', 7, 0], ['YCR072C', 8, 0], ['YPL146C', 10, 0], ['YKL052C', 10, 0], ['YDR201W', 16, 0], ['YOR078W', 27, 0], ['YPL217C', 13, 0], ['YMR128W', 16, 0], ['YBR247C', 58, 0], ['YHR148W', 11, 0],

In [55]:
def prob_escencial(g,kcutoff):
    g = asignar_esencialidad(g)
    PE=[]
    PE2=[]
    distribucion_de_grado=[]
    grados=[]
    
    for node in g.nodes:
        grados.append(g.degree(node))
    
    for k in range(max(list(grados))):
        r = 0
        for node in g.nodes:
            if g.degree(node) == k:
                r = r + 1
        distribucion_de_grado.append([k,r])
           
    #Se analizaron los nodos hasta grado k = 12 porque para grados mayores hay poca cantidad de nodos.
    for k in range(len(distribucion_de_grado)):
        if distribucion_de_grado[k][1]>kcutoff:
            cont_e = 0
            for node in g.nodes:
                if g.degree(node)==k and g.node[node]['escencialidad'] == "escencial":
                        cont_e = cont_e + 1

            PE.append([distribucion_de_grado[k][0],cont_e/distribucion_de_grado[k][1]])

    x = np.zeros (len(PE))
    y = np.zeros (len(PE))

    for i in range(len(PE)):
        x[i]=PE[i][0]
        y[i]=PE[i][1]

    return [x,y]

In [56]:
PEY2H = prob_escencial(gY2H,12)
PELIT = prob_escencial(gLIT,10)
PELITr = prob_escencial(gLITr,10)

In [57]:
print(PEY2H)
print(PELIT)
print(PELITr)

[array([1., 2., 3., 4., 5., 6., 7., 8., 9.]), array([0.22769231, 0.21980676, 0.19704433, 0.23529412, 0.1641791 ,
       0.21428571, 0.22857143, 0.42857143, 0.26315789])]
[array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13.]), array([0.26415094, 0.35793358, 0.45959596, 0.51666667, 0.47474747,
       0.55882353, 0.59677419, 0.58823529, 0.67567568, 0.52380952,
       0.77777778, 0.61538462, 0.5625    ])]
[array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21., 22., 23., 24., 26.]), array([0.00617076, 0.11374408, 0.15670103, 0.19032258, 0.24087591,
       0.24418605, 0.29054054, 0.36956522, 0.33035714, 0.38372093,
       0.51388889, 0.47761194, 0.42857143, 0.47368421, 0.5       ,
       0.51111111, 0.57142857, 0.57142857, 0.62162162, 0.66666667,
       0.45454545, 0.58823529, 0.34615385, 0.41176471, 0.64285714,
       0.54545455])]


In [63]:
def graficos_PE(g,PE,alfa,Beta):
    
    x=PE[0]
    y=np.log(1-PE[1])
    
    fitfunc = lambda p, x: p[0]*x+p[1] # Funcion con la que queremos fitear la data
    p0 = [-0.2, -0.2] # Parametros iniciales para fitear
    errfunc = lambda p, x, y: fitfunc(p, x) - y # Distancia a la función objetivo
    p1, success = optimize.leastsq(errfunc, p0[:], args=(x, y))

    #Alfa
    A=1-10**p1[0]
    #Beta
    B=1-10**p1[1]
    
    return [p1[0],p1[1],A,B]

In [113]:
ajusteY2H=graficos_PE(gY2H,PEY2H,alfaY2H,betaY2H)
ajusteLIT=graficos_PE(gLIT,PELIT,alfaLIT,betaLIT)
ajusteLITr=graficos_PE(gLITr,PELITr,alfaLITr,betaLITr)

print(ajusteY2H)
print(ajusteLIT)
print(ajusteLITr)


[-0.019587548835818022, -0.18420861950937586, 0.04410002298483251, 0.34567821418023115]
[-0.05667261143800029, -0.4113105621581598, 0.12233781128628962, 0.6121270994851988]
[-0.03023217344911629, -0.18888511328026614, 0.0672444827971651, 0.3526861687969749]


### Pares compartidos

In [77]:
def Pek(alfa,beta,k):
    Pe=1-(1-beta)*(1-alfa)**k
    return Pe

In [159]:
def pares_compartidos (g,alfa,beta,cota):
    g = asignar_esencialidad(g)
    ya_recorr = []
    N_pares_escen = 0
    N_pares_no_escen = 0
    escen = []
    PE=[]           
    
    #numero total de pares con >cota
    pares_totales=[]
    
    for node1 in g.nodes:
        ya_recorr.append(node1)
        for node2 in g.nodes:
            if node2 not in ya_recorr and node2 not in g.neighbors(node1):
                if len(list(nx.common_neighbors(g,node1,node2)))>cota:
                    #vecinos_comunes.append(list(nx.common_neighbors(g,node1,node2)))
                    pares_totales.append([node1,node2])
                    k1=len(list(g.neighbors(node1)))
                    k2=len(list(g.neighbors(node2)))
                    PE.append(Pek(alfa,beta,k1)*Pek(alfa,beta,k2)+(1-Pek(alfa,beta,k1))*(1-Pek(alfa,beta,k2)))
                    if g.node[node1]['escencialidad'] == 'escencial' and g.node[node2]['escencialidad'] == 'escencial':
                        N_pares_escen = N_pares_escen + 1

                    else:
                        if g.node[node1]['escencialidad'] == 'no escencial' and g.node[node2]['escencialidad'] == 'no escencial':
                            N_pares_no_escen = N_pares_no_escen + 1
#     for node1 in g.nodes:
#         for node2 in g.nodes:
#             if node1 != node2 and node2 not in nodo_ya_recorrido:
#                 if g.has_edge (node1,node2) is False:
#                     vec_node_1 = g.neighbors (node1)
#                     vec_node_2 = g.neighbors (node2)
#                     if len (set(vec_node_1) & set (vec_node_2)) >= 3:
#                         k1=g.degree(node1)
#                         k2=g.degree(node2)
#                         PE.append(Pek(alfa,beta,k1)*Pek(alfa,beta,k2)+(1-Pek(alfa,beta,k1))*(1-Pek(alfa,beta,k2)))
#                         if g.node[node1]['escencialidad'] == 'escencial' and g.node[node2]['escencialidad'] == 'escencial':
#                             N_pares_escen = N_pares_escen + 1

#                         elif g.node[node1]['escencialidad'] == 'no escencial' and g.node[node2]['escencialidad'] == 'no escencial':
#                             N_pares_no_escen = N_pares_no_escen + 1
    Total = len(pares_totales)
    x = [N_pares_escen,N_pares_no_escen]
    
    return [Total,x[0]+x[1],sum(PE)]


In [150]:
k1=3
k2=4
Pek(ajusteY2H[2],ajusteY2H[3],k1)*Pek(ajusteY2H[2],ajusteY2H[3],k2)+(1-Pek(ajusteY2H[2],ajusteY2H[3],k1))*(1-Pek(ajusteY2H[2],ajusteY2H[3],k2))

0.506624219586461

In [163]:
Pares_de_misma_especie_Y2H = pares_compartidos(gY2H,ajusteY2H[2],ajusteY2H[3],0)
Pares_de_misma_especie_Y2H

[23073, 15087, 11558.086367977921]

In [161]:
Pares_de_misma_especie_LIT = pares_compartidos(gLIT,ajusteLIT[2],ajusteLIT[3],2)
Pares_de_misma_especie_LIT

[730, 389, 560.7351151929096]

In [162]:
Pares_de_misma_especie_LITr = pares_compartidos(gLITr,ajusteLITr[2],ajusteLITr[3],2)
Pares_de_misma_especie_LITr

[10777, 6187, 7123.1299641604]

In [165]:
print((ajusteY2H[1]-ajusteY2H[2])/ajusteY2H[1])
print((ajusteLIT[1]-ajusteLIT[2])/ajusteLIT[1])
print((ajusteLITr[1]-ajusteLITr[2])/ajusteLITr[1])
print((1135-945)/1135)

1.2394026028873633
1.2974341593475724
1.356007319101894
0.16740088105726872
